In [1]:
import cv2 
import numpy as np
import pandas as pd
import time
import io
import os
import subprocess
from datetime import datetime

In [2]:
def showImage(title, img):
    cv2.imshow(title,img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [6]:
def skeleton(src):
    skel = np.zeros(src.shape,np.uint8)
    size = np.size(src)
    ret,img2 = cv2.threshold(src,127,255,0)
    element = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
    done = False
    
    while( not done):
        eroded = cv2.erode(img2,element)
        temp = cv2.dilate(eroded,element)
        temp = cv2.subtract(img2,temp)
        skel = cv2.bitwise_or(skel,temp)
        img2 = eroded.copy()
    
        zeros = size - cv2.countNonZero(img2)
        if zeros==size:
            done = True

    return skel

## Read video

In [32]:
cap = cv2.VideoCapture('./CLEANED_VIDEOS/' + 'adas.mp4')
if (cap.isOpened()== False): 
      print("Error opening video stream or file")
i = 0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        kernel = np.ones((1,1),np.uint8)
        #opening = cv2.morphologyEx(frame, cv2.MORPH_CLOSE, kernel)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # convert to grayscale
        
        #EQUALIZED GRAYSCALE
        clahe = cv2.createCLAHE(clipLimit=10.0, tileGridSize=(5,5))
        cl1 = clahe.apply(gray)
                
        #blur = cv2.blur(c, (5,5)) # blur the image
        thresh = cv2.adaptiveThreshold(cl1, 255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,15,2)
        
        #canny_output = cv2.Canny(thresh, 50, 150,apertureSize=3)
        skel = skeleton(thresh)
        kernel = np.ones((3,1),np.uint8)
        skel1 = cv2.dilate(skel, kernel, iterations=2)

        kernel = np.ones((1,3),np.uint8)
        skel2 = cv2.dilate(skel, kernel, iterations=2)

        skel3 = cv2.bitwise_or(skel1, skel2)
        
        # Find contours
        contours, hierarchy = cv2.findContours(skel, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        
        for cnt in contours:
            approx = []
            epsilon = 0.1*cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt, epsilon, True)
            #showImage('img', img)
            area = cv2.contourArea(cnt)
            if (area > 1000 and area < 500000 and  len(approx) == 4):
                i += 1
                cv2.drawContours(frame, [approx], -1, (15, 254, 251), 2)
                #cv2.imwrite('./CLEANED_VIDEOS/'  +'images/result_{}.png'.format(i),frame)
        cv2.imshow('frame',frame)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else: 
        break
cap.release()
cv2.destroyAllWindows()

In [4]:
kernel = np.ones((1,1),np.uint8)
opening = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
showImage('OPENING', opening)

NameError: name 'img' is not defined

## Transform

In [ ]:
gray = cv2.cvtColor(opening, cv2.COLOR_BGR2GRAY) # convert to grayscale
showImage('GRAY', gray)
blur = cv2.blur(gray, (5,5)) # blur the image
showImage('IBLUR', blur)
thresh = cv2.adaptiveThreshold(blur, 255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
showImage('THRESH', thresh)

## CLAHE (Contrast Limited Adaptive Histogram Equalization)

In [ ]:
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(3,3))
cl1 = clahe.apply(thresh)
showImage('CLASHE',cl1)

## Transform and find contours

In [ ]:
#src_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#src_gray = cv2.blur(src_gray, (3,3))
#showImage(src_gray)

# Detect edges using Canny
canny_output = cv2.Canny(cl1, 50, 150,apertureSize=3)
showImage('CANNY', canny_output)

# Find contours
contours, hierarchy = cv2.findContours(canny_output, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

## Draw contours

In [ ]:
#drawing = np.zeros((canny_output.shape[0], canny_output.shape[1], 3), dtype=np.uint8)
for cnt in contours:
    approx = []
    epsilon = 0.1*cv2.arcLength(cnt, True)
    approx = cv2.approxPolyDP(cnt, epsilon, True)
    #showImage('img', img)
    area = cv2.contourArea(cnt)

    if (area > 10000 and len(approx) == 4):
        print(area)
        print('cont', len(approx))
        cv2.drawContours(img, [approx], -1, (15, 254, 251), 2)
        showImage('CONTOURS', img)
        #break
    ##    
    #    x = approx.ravel()[0]
    #    y = approx.ravel()[1]
    #    cv2.putText(img, str(area), (x, y), cv2.FONT_HERSHEY_SIMPLEX , 1, (35))
    #    #cv2.imwrite(clean_videos_path  +'images/screenshot_res.png',img)


## Equalization image

In [198]:
equ = cv2.equalizeHist(img)
showImage('INITIAL IMAGE', img)
showImage('EQUALIZATION',equ)

error: OpenCV(4.0.0) /io/opencv/modules/imgproc/src/histogram.cpp:3334: error: (-215:Assertion failed) _src.type() == CV_8UC1 in function 'equalizeHist'


In [ ]:
img = cv2.GaussianBlur(img, (11, 11), 0)
sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0)
sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1)
laplacian = cv2.Laplacian(img, cv2.CV_64F, ksize=5)
canny = cv2.Canny(img, 100, 150)
cv2.imshow("Image", img)
cv2.imshow("Sobelx", sobelx)
cv2.imshow("Sobely", sobely)
cv2.imshow("Laplacian", laplacian)
cv2.imshow("Canny", canny)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
#for file in files[:1]:
file = 'canal12__03-05-2019__09-30.avi'
cap = cv2.VideoCapture(clean_videos_path + file)
i = 0
while(cap.isOpened()):
    i += 1
    ret, frame = cap.read()
    if ret == True:
        # Convert image to gray and blur it
        src_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        src_gray = cv2.blur(src_gray, (3,3))
        
        # Detect edges using Canny
        canny_output = cv2.Canny(src_gray, 100, 100 * 2)
        # Find contours
        contours, hierarchy = cv2.findContours(canny_output, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        # Draw contours
        #drawing = np.zeros((canny_output.shape[0], canny_output.shape[1], 3), dtype=np.uint8)
        for cnt in contours:
            approx = cv2.approxPolyDP(cnt, 0.01*cv2.arcLength(cnt, True), True)
            
            area = cv2.contourArea(cnt)
            if len(approx) == 4 and area > 20:
                cv2.drawContours(frame, [approx], 0, (0), 2)
                x = approx.ravel()[0]
                y = approx.ravel()[1]
                cv2.putText(frame, str(area), (x, y), cv2.FONT_HERSHEY_SIMPLEX , 3, (2))
                cv2.imwrite(clean_videos_path  +'images/result_{}.png'.format(i),frame)
                cv2.imshow("shapes", frame)
            #cv2.imshow("Threshold", frame)
        # Show in a window
        #cv2.imshow('Contours', drawing)
        #Press Q on keyboard to  exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    # Break the loop
    else: 
        break

cap.release()
cv2.destroyAllWindows()
